## Example of custom callback using HoloViews linked streams

**Authors:** Matias C. Kind, Wil O'Mullane, Angelo Fausti

**Stack Version:** w_2018_22

**Image Size:** Small

**Purpose:**  Demonstrates the Holoviews+Datashader+Bokeh stack for data exploration in particular the  [linked streams](http://holoviews.org/user_guide/Custom_Interactivity.html) functionality.

In [ ]:
import os
from lsst.daf.persistence import Butler
import pandas as pd
import holoviews as hv
import numpy as np
from holoviews.operation.datashader import datashade
import param
from holoviews import streams

hv.extension('bokeh')

We'll use results from the [HSC RC2 reprocessing](https://confluence.lsstcorp.org/display/DM/Reprocessing+of+the+HSC+RC2+dataset).

In [ ]:
datadir = '/datasets/hsc/repo/rerun/RC/w_2018_15/DM-14123/'
butler = Butler(datadir)

Which dataset types are available? which filters, tracts, and patches can we use? 

Currently, it is not possible to query the data repository for coadd metadata. For now we have to know in advance the dataIds available using information from the link above. Butler Gen3 will fix that.

In [ ]:
# dataids = butler.subset('deepCoadd_forced_src', dataId={'filter': 'HSC-G', 'tract': 9813 }).cache

Let's read the `deepCoadd_forced_src` dataset type for the `HSC-G` filter for one tract and one patch. We'll use pandas dataframes which plays well with HoloViews. 



In [ ]:
_filter = 'HSC-G'
tract = 9813
patch = '7,3'

df = butler.get('deepCoadd_forced_src', 
                dataId={'tract': tract, 'patch': patch, 'filter': _filter}).asAstropy().to_pandas()

# Get only a few columns of interest
df = df[['coord_ra', 'coord_dec', 'modelfit_CModel_flux']]



In [ ]:
df['modelfit_CModel_flux'].describe()

Get rid of `NaN` values and negative fluxes. 

In [ ]:
df = df.dropna()
df = df.loc[df['modelfit_CModel_flux'] > 0]

In [ ]:
print('Total objects in this patch = {}'.format(df.size))

By creating a HoloViews dataset we annotate the dataset for visualization declaring which columns correspond to independent variables (called key dimensions or kdims) and dependent variables (called value dimensions or vdims). 



In [ ]:
kdims = [('coord_ra', 'RA(deg)'), ('coord_dec', 'Dec(deg)')]
vdims = [('modelfit_CModel_flux', 'CModel Flux')]
ds = hv.Dataset(df, kdims, vdims)
ds

In order to visualize a HoloViews dataset we can select one of the [Holoviews elements](http://holoviews.org/reference/index.html#elements). Perhaps the most obvious visualization for this dataset is to display the spatial distribution of the objects using the Point element. 

In [ ]:
%%opts Points [tools=['box_select']]
points = hv.Points(ds).options({'Points': {'size': 1, 'alpha': 0.5}})
points

`hv.Points` works well with datasets size (`N<~50k`). Later we'll load a tract worth of data and we'll need another approach for visualization, called datashader.

For this dataset, datashader is not really useful. 

In [ ]:
%%opts Points [tools=['box_select']]
image = datashade(hv.Points(ds))
image

Notice, however, that the `box_select` tool is not available when using datashader. That's because a selection on the image representation cannot return the index of the data points anymore. That means we'll have to use another strategy for selecting data when using datashader later.


Creating a histrogram is a little more involving, apparently it is not possible to map directly the value dimension onto the Histogram element, and reuse the label annotations for this variable yet.

In [ ]:
frequencies, edges = np.histogram(ds.data['modelfit_CModel_flux'])
hist = hv.Histogram((np.log(frequencies), edges))
hist

Now we can use the [linked streams](http://holoviews.org/user_guide/Custom_Interactivity.html) functionality to link both plots. 

In [ ]:
listing = ', '.join(sorted([str(s.name) for s in param.descendents(streams.LinkedStream)]))
listing

There are streams to access plotting selections made using box- and lasso-select tools (`Selection1D`), the plot size (`PlotSize`) and the `Bounds` of a selection. This is really exciting, we can select regions on the spatial plot above and that will produce an histogram for the selected objects.

With `hv.Points` we can use the`Selection1D` linked stream and declare the `points` instance as the source of selection. 

In [ ]:
selection = streams.Selection1D(source=points)
print('The %s stream has contents %r' % (selection, selection.contents))

We have this custom callback function that will create an histogram based on the selected data.

In [ ]:
def update_histogram(index):
    # this will initialize the histogram with data in case there's no selection
    if len(index) > 0:
        selected_flux = ds.data.iloc[index]['modelfit_CModel_flux']
    else:
        selected_flux = ds.data['modelfit_CModel_flux']
    
    frequencies, edges = np.histogram(selected_flux)
    
    hist = hv.Histogram((np.log(frequencies), edges))
    return hist
    

Finally we associate the selection stream instance to the callback function. This is done using a `DynamicMap`.

Use the `Box Select` tool to select an area in the RA, Dec plot and see the histogram update.


In [ ]:
%%opts Points [tools=['box_select']]
dmap = hv.DynamicMap(update_histogram, streams=[selection])
points + dmap

The following cells are useful for debugging:

In [ ]:
# selection.contents

In [ ]:
#index = selection.contents['index']

#frequencies, edges = np.histogram(ds.data.iloc[index]['modelfit_CModel_flux'], 20)
#hist = hv.Histogram((frequencies, edges))
#hist

Now, lets load a tract worth of data to make the datashader example more interesting.

In [ ]:
_filter = 'HSC-G'
tract = 9813

Because of the limitations mentioned bove in the butler implementation, we'll have to look at the data output repository to get the list of the patches available. 

Then we can loop over the dataIds and load the corresponding catalogs.

In [ ]:
patches = [x[1] for x in os.walk("{}/deepCoadd-results/{}/{}".format(datadir, _filter, tract))][0]

In [ ]:
%%time
catalogs = []
for patch in patches:
    print("Loading filter: {}, tract: {}, patch: {}".format(_filter, tract, patch))
    catalog = butler.get('deepCoadd_forced_src', 
                         dataId={'tract': tract, 'patch': patch, 'filter': _filter}).asAstropy().to_pandas()
    # Get only a few columns of interest
    tmp = catalog[['coord_ra', 'coord_dec', 'modelfit_CModel_flux']]
    catalogs.append(tmp)


In [ ]:
df = pd.concat(catalogs)
df = df.dropna()
df = df.loc[df['modelfit_CModel_flux'] > 0]
print('Total objects in this tract = {}'.format(df.size))

As mentioned above we need datashader to visualize a dataset of this size. See how nice it plays with Bokeh pan and zomm tools.

In [ ]:
kdims = [('coord_ra', 'RA(deg)'), ('coord_dec', 'Dec(deg)')]
vdims = [('modelfit_CModel_flux', 'CModel Flux')]
ds = hv.Dataset(df, kdims, vdims)
ds

In [ ]:
%%opts Points [tools=['box_select']]
image = datashade(hv.Points(ds))
image

In [ ]:
frequencies, edges = np.histogram(ds.data['modelfit_CModel_flux'])
hist = hv.Histogram((np.log(frequencies), edges))
hist

Let's use a stream linked to the axis limits (RangeXY)

In [ ]:
x_range = (np.min(ds.data['coord_ra']), np.max(ds.data['coord_ra']))
y_range = (np.min(ds.data['coord_dec']), np.max(ds.data['coord_dec']))

rangexy = streams.RangeXY(source=image, x_range=x_range, y_range=y_range)
print('The %s stream has contents %r' % (rangexy, rangexy.contents))

Now our callback function must accept the `x_range`, `y_range` arguments that are used for slicing the pandas dataframe.

In [ ]:
def update_histogram(x_range, y_range):
    
    tmp = ds.data.loc[(ds.data['coord_ra'] > x_range[0]) & 
                      (ds.data['coord_ra'] < x_range[1]) &
                      (ds.data['coord_dec'] > y_range[0]) &
                      (ds.data['coord_dec'] < y_range[1])]
   
    selected_flux = tmp['modelfit_CModel_flux']
    
    frequencies, edges = np.histogram(selected_flux)
    
    hist = hv.Histogram((np.log(frequencies), edges))
    return hist
    

Associate the `rangexy` stream instance to the callback function with a `DynamicMap`.

Use the `Wheel Zoom` tool to nagivate through the RA, Dec plot and see the histogram update.


In [ ]:
dmap = hv.DynamicMap(update_histogram, streams=[rangexy])
image + dmap

## Another approach to selection with datashader

Keith

The variation below shows how to perform a selection on a datashaded 2D histogram with a linked plot or callback to get the objects that pass the selection

In [248]:
image2 = datashade(hv.Points(ds))
bounds_default = (np.min(ds.data['coord_ra']), np.min(ds.data['coord_dec']), np.max(ds.data['coord_ra']), np.max(ds.data['coord_dec']))
bounds = streams.BoundsXY(source=image2, bounds=bounds_default)

In [249]:
def update_selection(bounds=bounds):

    selection = (ds.data['coord_ra'] > bounds[0]) \
    & (ds.data['coord_ra'] < bounds[2]) \
    & (ds.data['coord_dec'] > bounds[1]) \
    & (ds.data['coord_dec'] < bounds[3])
    tmp = ds.data.loc[selection]

    selected_flux = tmp['modelfit_CModel_flux']
    
    frequencies, edges = np.histogram(selected_flux)
    
    hist2 = hv.Histogram((np.log(frequencies), edges))
    return hist2

In [250]:
dmap2 = hv.DynamicMap(update_selection, streams=[bounds])
image2 + dmap2

/opt/lsst/software/stack/python/miniconda3-4.3.21/lib/python3.6/site-packages/ipykernel/__main__.py:10: RuntimeWarning: divide by zero encountered in log


:Layout
   .DynamicMap.I  :DynamicMap   []
      :RGB   [coord_ra,coord_dec]   (R,G,B,A)
   .DynamicMap.II :DynamicMap   []
      :Histogram   [x]   (Frequency)

In [251]:
selection = (ds.data['coord_ra'] > bounds.bounds[0]) \
    & (ds.data['coord_ra'] < bounds.bounds[2]) \
    & (ds.data['coord_dec'] > bounds.bounds[1]) \
    & (ds.data['coord_dec'] < bounds.bounds[3])
tmp = ds.data.loc[selection]
   
image_selection = datashade(hv.Points(tmp))
image_selection

:DynamicMap   []
   :RGB   [coord_ra,coord_dec]   (R,G,B,A)